In [38]:
import pandas as pd
import numpy as np

import tensorflow as tf
import os
import PIL
import shutil

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Add, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [39]:
# base path
base_dir = 'D:/archive/Humans/'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

'''# training data path
train_Kang_dir = os.path.join(train_dir, 'Kang')
train_Choi_dir = os.path.join(train_dir, 'Choi')
train_Hond_dir = os.path.join(train_dir, 'Hong')
'''
train_face_dir = os.path.join(train_dir, 'face')

'''
# validation data path
validation_Kang_dir = os.path.join(validation_dir, 'Kang')
validation_Choi_dir = os.path.join(validation_dir, 'Choi')
valdiation_Hong_dir = os.path.join(validation_dir, 'Hong')'''
validation_face_dir = os.path.join(validation_dir, 'face')

In [49]:
train_datagen = ImageDataGenerator(rescale= 1./255,
                                   rotation_range=25,
                                   width_shift_range=0.05,
                                   height_shift_range=0.05,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=16,
                                                    color_mode='rgb',
                                                    class_mode='sparse',
                                                    target_size=(224,224))
validation_generator = validation_datagen.flow_from_directory(validation_dir,
                                                              batch_size=4,
                                                              color_mode='rgb',
                                                              class_mode='sparse',
                                                              target_size=(224,224))



Found 5463 images belonging to 1 classes.
Found 1756 images belonging to 1 classes.


In [50]:
train_generator.class_indices

{'face': 0}

# Model

In [51]:
'''def residual_block(x, filters, kernel_size=3, stride=1):
    shortcut = x
    
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.activations.relu(x)
    
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = Add()([x, shortcut])
    x = tf.keras.activations.relu(x)
    return x

input_tensor = Input(shape=(150,150,3))

x = Conv2D(64, (7,7), padding="same", strides=(2,2), activation='relu')(input_tensor)
x = MaxPooling2D((3,3), strides=(2,2), padding="same")(x)

x = residual_block(x,64)
x = residual_block(x,64)

x = GlobalAveragePooling2D()(x)

output_tensor = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_tensor, outputs=output_tensor)

model.summary()'''

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))

x = GlobalAveragePooling2D()(base_model.output)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model. compile(optimizer=Adam(),
               loss = SparseCategoricalCrossentropy(),
               metrics=['accuracy'])

history = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch=4,
                    epochs=50,
                    validation_steps=4,
                    verbose=2)

train_loss, train_accuracy = model.evaluate(train_generator)
validation_loss, validation_accuracy = model.evaluate(validation_generator)
print(f"Training Loss: {train_loss}, Training Accuracy: {train_accuracy}")
print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")


Epoch 1/50
4/4 - 42s - loss: 0.0000e+00 - accuracy: 0.8438 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - 42s/epoch - 10s/step
Epoch 2/50
4/4 - 16s - loss: 0.0000e+00 - accuracy: 0.7656 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - 16s/epoch - 4s/step
Epoch 3/50
4/4 - 17s - loss: 0.0000e+00 - accuracy: 0.7969 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - 17s/epoch - 4s/step
Epoch 4/50
4/4 - 17s - loss: 0.0000e+00 - accuracy: 0.7500 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - 17s/epoch - 4s/step
Epoch 5/50
4/4 - 17s - loss: 0.0000e+00 - accuracy: 0.7812 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - 17s/epoch - 4s/step
Epoch 6/50
4/4 - 16s - loss: 0.0000e+00 - accuracy: 0.8281 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - 16s/epoch - 4s/step
Epoch 7/50
4/4 - 16s - loss: 0.0000e+00 - accuracy: 0.8125 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00 - 16s/epoch - 4s/step
Epoch 8/50
4/4 - 16s - loss: 0.0000e+00 - accuracy: 0.7656 - val_loss: 0.0000e+00 

In [11]:
model.save('face_detect.h5')